# Extracting Total Load 


Import modules

In [86]:
%matplotlib inline
import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import scipy
from scipy.stats import zscore

Set up participant number

In [87]:
participantNumber = ['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14',
                     'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28',
                     'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35']

Set up block number

In [88]:
blockNumber = ['B1', 'B2', 'B3']

Set up fabric code

In [89]:
fabricCode = ['A', 'B', 'C']


Set up episode number

In [90]:
episodeNumber = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0]

Read in compilation table

In [91]:
df = pd.read_csv('C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Total load calculations\CompilationTable.csv')

Remove irrelevent coloums from dataframe

In [92]:
df = df.drop(
    ['Position_mm', 'File_Number', 'Reference', 'Protocol', 'Analysis', 'Date_Stamp', 'Time_Stamp', 'DataSaveBlock',
     'BlockStamp_s', 'BrickInBlock', 'Time_s', 'Adjustment_s', 'RelativeTime_s', 'Trigger_V', 'SensorA_Abs_V',
     'SensorB_Abs_V', 'SensorA_Net_V', 'SensorB_Net_V', 'ForceAtXminus47p5mm_g', 'ForceAtXplus47p5mm_g', 'Study'],
    axis=1)


Filter data

In [93]:
dfFiltered = df.loc[~(df['Episode_Number'] ==0)]
dfFiltered = dfFiltered[dfFiltered['EpisodeRelativeTime_s'] >= 0]
dfFiltered= dfFiltered[dfFiltered['EpisodeRelativeTime_s'] < 6]
dfFiltered = dfFiltered[dfFiltered['TotalLoad_g'] > 1]
#dfFiltered = dfFiltered.reset_index()
#dfFiltered

Create empty data frames

In [94]:
# Create empty data frame to append episode means to
dfEpisodes = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Episode Number', 'Mean Load'])

# Create empty  data frame to append block means to
dfFabric = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Mean Load'])

#Create empty data frame to append Participant means to
dfParticipant = pd.DataFrame(columns=['Participant', 'Fabric', 'Mean Load'])

Loop to calculate mean for each episode

In [95]:

for p in participantNumber:
    dfP = dfFiltered[dfFiltered['Participant'] == p]  # when value in 'Participant' column is equal to the participant value in loop
    for b in blockNumber:
        dfB = dfP[dfP['Study_Block'] == b]  # when value in 'Study_Block' column is equal to theStudy_block value in loop
        dfStatsFabric = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Mean Load'])
        for f in fabricCode:
            dfF = dfB[dfB['Fabric'] == f]  # when value in 'Fabric' column is equal to the fabric value in loop
            dfStatsEpisodes = pd.DataFrame(columns=['Participant', 'Block', 'Fabric', 'Mean Load'])
            for e in episodeNumber:
                dfE = dfF[dfF['Episode_Number'] == e]  # when value in 'Episode number' column is equal to the episode value in loop
                dfE = dfE.assign(TotalLoad_g_zscore = '') #create a new empty column in data fram dfE called 'TotalLoad_g_zscore'
                dfE['TotalLoad_g_zscore'] = zscore(dfE['TotalLoad_g']) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
                dfFiltered_zscore = dfE.loc[~(dfE['TotalLoad_g_zscore'] <= -2)] #Filter out zscores that are less than -2
                dfFiltered_zscore = dfFiltered_zscore.loc[~(dfFiltered_zscore['TotalLoad_g_zscore'] >= 2)] #Filter out scores that are great than 2
                #dfFiltered_zscoreMax = dfFiltered_zscore['TotalLoad_g'].max()
                #dfFiltered_zscoreMax = dfFiltered_zscoreMax *1.5
                meanLoad = dfFiltered_zscore['TotalLoad_g'].mean()
                dfStatsEpisodes = dfStatsEpisodes.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'Mean Load': meanLoad},ignore_index=True)
                dfEpisodes = dfEpisodes.append({'Participant': p, 'Block': b, 'Fabric': f, 'Episode Number': float(e), 'Mean Load': meanLoad},ignore_index=True)
            
            # Create plot for each episode number for the block and fabric
            #plt.scatter(dfStatsEpisodes_zscore['Episode Number'], dfStatsEpisodes_zscore['Mean Load'])
            #plt.title(str(p)+'_'+str(b)+'_'+str(f))
            #plt.ylim(0, dfFiltered_zscoreMax)
            #plt.xticks(episodeNumber)
            #plt.show()
            #input("Press Enter to continue...")



Remove NaN values from episodes data frame

In [96]:
dfEpisodesFiltered = dfEpisodes.dropna()


Loop to calculate mean for each block

In [97]:
for p in participantNumber: 
    dfP3 = dfEpisodesFiltered[dfEpisodesFiltered['Participant'] == p]
    for b in  blockNumber: 
        dfB3 = dfP3[dfP3['Block'] == b]
        for f in fabricCode: 
            dfF3 = dfB3[dfB3['Fabric'] == f]
            dfF3 = dfF3.assign(Mean_Load_zscore = '') #create a new empty column in data fram dfE called 'TotalLoad_g_zscore'
            dfF3['Mean_Load_zscore'] = zscore(dfF3['Mean Load']) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
            dfF3Filtered_zscore = dfF3.loc[~(dfF3['Mean_Load_zscore'] <= -2)] #Filter out zscores that are less than -2
            dfF3Filtered_zscore = dfF3Filtered_zscore.loc[~(dfF3Filtered_zscore['Mean_Load_zscore'] >= 2)] #Filter out scores that are great than 2
            meanLoadFabric = dfF3Filtered_zscore['Mean Load'].mean()
     
            #Append mean load for each fabric for each block to a dataframe
            dfFabric = dfFabric.append({'Participant': p, 'Block': b, 'Fabric': f, 'Mean Load': meanLoadFabric},ignore_index=True)
            

Remove NaN values from fabric data frame 

In [98]:
dfFabricFiltered = dfFabric.dropna()

Loop to caluclate mean for each fabric

In [99]:

for p in participantNumber:
    dfP4 = dfFabricFiltered[dfFabricFiltered['Participant'] == p]
    for f in fabricCode: 
        dfF4 = dfP4[dfP4['Fabric'] == f]
        dfF4 = dfF4.assign(Mean_Load_zscore = '') #create a new empty column in data fram dfE called 'TotalLoad_g_zscore'
        dfF4['Mean_Load_zscore'] = zscore(dfF4['Mean Load']) # create zscore for 'TotalLoad_g' and put values in a new column of dfE named 'TotalLoad_g_zscore'
        dfF4Filtered_zscore = dfF4.loc[~(dfF4['Mean_Load_zscore'] <= -2)] #Filter out zscores that are less than -2
        dF4Filtered_zscore = dfF4Filtered_zscore.loc[~(dfF4Filtered_zscore['Mean_Load_zscore'] >= 2)] #Filter out scores that are great than 2
        meanLoadBlock = dfF4Filtered_zscore['Mean Load'].mean()
            
        #Append mean load for each fabric for each block to a dataframe
        dfParticipant = dfParticipant.append({'Participant': p, 'Fabric': f, 'Mean Load':meanLoadBlock},ignore_index=True)
                           


Export data

In [100]:
exportParticipant= dfParticipant.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Total load calculations\Fabric_Means.csv', mode='a', index=None)
exportFabric= dfFabric.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Total load calculations\Block_Means.csv', mode='a', index=None)
exportEpisodes= dfEpisodes.to_csv(r'C:\LINEAR_SENSOR\_Processed_Data\_MATLAB_Processed\Total load calculations\Episode_Means.csv', mode='a', index=None)